# Gensim

**Gensim** is billed as a Natural Language Processing package that does ‘Topic Modeling for Humans’. But it is practically much more than that. It is a leading and a state-of-the-art package for processing texts, working with *word vector models* and for building *topic models*.

But the width and scope of facilities to build and evaluate topic models are unparalleled in gensim, plus many more convenient facilities for text processing.

Also, another significant advantage with gensim is: it lets you handle large text files without having to load the entire file in memory.

In [ ]:
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

## Dictionary and Corpus

**Dictionary** is an object that maps each word to a unique id.

The dictionary object is typically used to create a ‘bag of words’ Corpus. It is this Dictionary and the bag-of-words (Corpus) that are used as inputs to topic modeling and other models that Gensim specializes in.

*gensim.utils.simple_preprocess* Convert a document into a list of lowercase tokens, ignoring tokens that are too short or too long.

In [ ]:
# from a list of sentences

documents = ["If you use a car frequently, the first step to cutting",
             "down your emissions may well be to simply", 
             "fully consider the", 
             "alternatives available to you."
             ]

# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in documents]

# Create dictionary
dictionary = corpora.Dictionary(texts)

print(dictionary)
print(dictionary.token2id)

With simple_preprocess

In [ ]:
dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in documents)
print(dictionary)
print(dictionary.token2id)

In [ ]:
#from document
dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in open('sample.txt', encoding='utf-8'))

**Corpus** object that contains the word id and its frequency in each document.

The dictionary object is typically used to create a ‘bag of words’ Corpus. It is this Dictionary and the bag-of-words (Corpus) that are used as inputs to topic modeling and other models that Gensim specializes in.

- Bag of words

In [ ]:
my_docs = ["Who let the dogs out?",
           "Who? Who? Who? Who?"]

# Tokenize the docs
tokenized_list = [simple_preprocess(doc) for doc in my_docs]

# Create the Corpus
dictionary = corpora.Dictionary()

#allow_update=True - add new words to dictionary
bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in tokenized_list]
print(bow_corpus)

print("Dictionary: ", dictionary.token2id)

The (0, 1) in line 1 means, the word with id=0 appears once in the 1st document.
Likewise, the (4, 4) in the second list item means the word with id 4 appears 4 times in the second document. And so on.

- TfIdf

In [ ]:
from gensim import models
import numpy as np

documents = ["This is the first line",
             "This is the second sentence",
             "This third document"]

# Create the Dictionary and Corpus
mydict = corpora.Dictionary([simple_preprocess(line) for line in documents])
corpus = [mydict.doc2bow(simple_preprocess(line)) for line in documents]

# Show the Word Weights in Corpus
for doc in corpus:
    print([[mydict[id], freq] for id, freq in doc])
print()

# Create the TF-IDF model
tfidf = models.TfidfModel(corpus, smartirs='ntc')

# Show the TF-IDF weights
for doc in tfidf[corpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])

Save and load

In [ ]:
# Save the Dict and Corpus
dictionary.save('mydict.dict')  # save dict to disk
corpora.MmCorpus.serialize('bow_corpus.mm', bow_corpus)  # save corpus to disk

In [ ]:
# Load them back
loaded_dict = corpora.Dictionary.load('mydict.dict')

corpus = corpora.MmCorpus('bow_corpus.mm')
for line in corpus:
    print(line)

## Datasets

Gensim provides an inbuilt API to download popular text datasets and word embedding models.

A comprehensive list of available datasets and models is maintained [here](https://raw.githubusercontent.com/RaRe-Technologies/gensim-data/master/list.json).

Using the API to download the dataset is as simple as calling the ```api.load()``` method with the right data or model name.

In [ ]:
import gensim.downloader as api

Short description

In [ ]:
api.info('text8')
api.info('glove-wiki-gigaword-50')

Dataset

In [ ]:
dataset = api.load("text8")
data = [d for d in dataset]
print(data[0])

Pretrained model

In [ ]:
w2v_model = api.load("glove-wiki-gigaword-50")
w2v_model.most_similar('blue')

## Word2Vec

A word embedding model is a model that can provide numerical vectors for a given word. Using the Gensim’s downloader API, you can download pre-built word embedding models like word2vec, fasttext, GloVe and ConceptNet. These are built on large corpuses of commonly occurring text data such as wikipedia, google news etc.

The training algorithms in the Gensim package were actually ported from the original Word2Vec implementation by Google and extended with additional functionality.

This module implements the word2vec family of algorithms, using *highly optimized* C routines, data streaming and Pythonic interfaces.

**Parameters:**

- ```sentences``` - (iterable of iterables, optional) – The sentences iterable can be simply a *list of lists of tokens*, but for larger corpora, consider an *iterable* that streams the sentences directly from disk/network.
- ```corpus_file``` (str, optional) – Path to a corpus file in LineSentence format. You may use this argument instead of sentences to get performance boost. *Only one of sentences or corpus_file arguments need to be passed* 
- ```size``` = 100 - Dimensionality of the word vectors.
- ```window``` = 5 - Maximum distance between the current and predicted word within a sentence.
- ```min_count``` = 5 (int, optional) – Ignores all words with total frequency lower than this.
- ```workers``` = 3 (int, optional) – Use these many worker threads to train the model (=faster training with multicore machines).
- ```sg``` = 0 ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
- ```hs``` = 0 ({0, 1}, optional) – If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
- ```negative``` = 5 (int, optional) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
- ```max_vocab_size``` = None (int, optional) Limits the RAM during vocabulary building; if there are more unique words than this, then prune the infrequent ones.
- ```iter``` (int, optional) – Number of iterations (epochs) over the corpus.

In [ ]:
import logging 
# Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
dataset = api.load("text8")
data = [d for d in dataset]

# Train Word2Vec model
model = Word2Vec(data)

```Word2Vec``` without ```sentences``` or ```corpus``` is initialization only, should be trained

In [ ]:
model = Word2Vec()
model.train(data)

**Save and load model**

In [ ]:
model.save('w2v_newmodel')
model = Word2Vec.load('w2v_newmodel')

You can **continue trainig**

In [ ]:
model.train([["hello", "world"]], total_examples=1, epochs=1)

**Word2vec input**


1) Parameter ```sentences```

Gensim’s word2vec expects a sequence of sentences as its input. Each sentence a list of words

   1.1 *List of list of tokens*

In [ ]:
input1 = [['first', 'sentence'], ['second', 'sentence']]
model1 = Word2Vec(input1, min_count=1)

1.2 Gensim only requires that the input must provide sentences sequentially, when iterated over. *No need to keep everything in RAM*

In [ ]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
                
input2 = MySentences('/some/directory') # a memory-friendly iterator
model = gensim.models.Word2Vec(input2)

See [BrownCorpus](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.BrownCorpus), [Text8Corpus](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus) or [LineSentence](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) in word2vec module for such examples.

BrownCorpus and Text8Corpus were implemented special for BrownCorpus and Text8 datasets. Text8 corpus, for example, consists of one line of cleaned and joined wikipedia articles.

LineSentence iterate over a file that contains sentences: one line = one sentence. Words must be already preprocessed and separated by whitespace.

In [ ]:
from gensim.test.utils import datapath
from gensim.models.word2vec importLineSentence

input3 = LineSentence(datapath('lee_background.cor'))
model = gensim.models.Word2Vec(input3)

2. Parameter ```corpus_file``` - path to a corpus file in LineSentence format

If corpus is in right format, parameter corpus_file may be passed instead of last cell

## Exploring the model

- Extract the trained word vectors from model.wv:

In [ ]:
model.wv['topic']

- Similarity

In [ ]:
model.wv.most_similar(positive = ['topic'])

In [ ]:
print('Similarity between (walk, walking): ', model.wv.similarity('walk', 'walking'))
print('Similarity between (duck, ducks): ', model.wv.similarity('duck', 'ducks'))
print('Similarity between (banana, pear): ', model.wv.similarity('banana', 'pear'))
print()
print('Similarity between (banana, sky): ', model.wv.similarity('banana', 'sky'))
print('Similarity between (walk, lie): ', model.wv.similarity('walk', 'lie'))
print('Similarity between (dark, slow): ', model.wv.similarity('dark', 'slow'))

- Analogy

In [ ]:
model.wv.most_similar(positive = ['lower', 'tall'], negative = ['low'])

In [ ]:
model.wv.most_similar(positive = ['mother', 'man'], negative = ['woman'])

- Matching

In [ ]:
print(model.wv.doesnt_match(['car', 'airplane', 'bed']), " doesn't match to [car, airplane]")
print(model.wv.doesnt_match(['red', 'blue', 'roof']), " doesn't match to [red, blue]")

## Compare with other pretrained embeddings

In [ ]:
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')
word2vec_model300 = api.load('word2vec-google-news-300')
glove_model300 = api.load('glove-wiki-gigaword-300')

To define which one performs better using the respective model's evaluate_word_analogies() 

Compute performance of the model on an analogy test set. The accuracy is reported (printed to log and returned as a score) for each section separately, plus there’s one aggregate summary at the end. 

Input:
- ```analogies``` (str) – Path to file, where lines are 4-tuples of words, split into sections by “: SECTION NAME” lines. See [this file](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/datasets/questions-words.txt) as example.

Output:
- ```score``` (float) – The overall evaluation score on the entire evaluation set

- ```sections``` (list of dict of {str : str or list of tuple of (str, str, str, str)}) – Results broken down by each section of the evaluation set. Each dict contains the name of the section under the key ‘section’, and lists of correctly and incorrectly predicted 4-tuples of words under the keys ‘correct’ and ‘incorrect’.

In [ ]:
f = open('questions-words.txt', 'r')
for i in range(5):
    print(f.readline())
f.close()

In [ ]:
word2vec_model300.evaluate_word_analogies(analogies='questions-words.txt')[0]

# fasttext_accuracy
fasttext_model300.evaluate_word_analogies(analogies='questions-words.txt')[0]

# GloVe accuracy
glove_model300.evaluate_word_analogies(analogies='questions-words.txt')[0]

## Doc2Vec

Unlike Word2Vec, a Doc2Vec model provides a vectorised representation of a group of words taken collectively as a single unit. It is not a simple average of the word vectors of the words in the sentence.

The training data for ```Doc2Vec``` should be a list of ```TaggedDocuments```. To create one, we pass a list of words and a unique integer as input to the ```models.doc2vec.TaggedDocument()```.

In [ ]:
#prepare dataset
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield TaggedDocument(list_of_words, [i])

dataset = api.load("text8")
data = [d for d in dataset]

train_data = list(create_tagged_document(data))
print(train_data[:1])

In [ ]:
#Train model
model = Doc2Vec(vector_size=50, min_count=1, epochs=40)
model.build_vocab(train_data)
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
#Get document vector
print(model.infer_vector('gensim is really awesome'.split(' ')))

Used materials: 
    
- https://www.machinelearningplus.com/nlp/gensim-tutorial/
- https://radimrehurek.com/gensim/models/word2vec.html
- https://radimrehurek.com/gensim/models/keyedvectors.html
- https://rare-technologies.com/word2vec-tutorial/